# Clustering Crypto

In [243]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [244]:
# Load the crypto_data.csv dataset.
file_path = "crypto_data.csv"
crypto_df = pd.read_csv(file_path)

print(crypto_df.shape)


(1252, 7)


In [245]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df[crypto_df.IsTrading == True]
crypto_df.head()

Unnamed: 0   CoinName Algorithm  IsTrading ProofType  TotalCoinsMined  \
0         42    42 Coin    Scrypt       True   PoW/PoS     4.199995e+01   
1        365    365Coin       X11       True   PoW/PoS              NaN   
2        404    404Coin    Scrypt       True   PoW/PoS     1.055185e+09   
3        611  SixEleven   SHA-256       True       PoW              NaN   
4        808        808   SHA-256       True   PoW/PoS     0.000000e+00   

  TotalCoinSupply  
0              42  
1      2300000000  
2       532000000  
3          611000  
4               0

In [246]:
print(crypto_df.shape)

(1144, 7)


In [247]:
# Keep all the cryptocurrencies that have a working algorithm.
##crypto_df = crypto_df[crypto_df['TotalCoinSupply'] != '0']
##print(crypto_df.shape)

In [248]:
# Remove the "IsTrading" column. 
crypto_df.drop(['IsTrading'],inplace= True,  axis=1)

print(crypto_df.shape)
crypto_df.head()

(1144, 6)


Unnamed: 0   CoinName Algorithm ProofType  TotalCoinsMined TotalCoinSupply
0         42    42 Coin    Scrypt   PoW/PoS     4.199995e+01              42
1        365    365Coin       X11   PoW/PoS              NaN      2300000000
2        404    404Coin    Scrypt   PoW/PoS     1.055185e+09       532000000
3        611  SixEleven   SHA-256       PoW              NaN          611000
4        808        808   SHA-256   PoW/PoS     0.000000e+00               0

In [249]:
# Remove rows that have at least 1 null value.
crypto_df = crypto_df.dropna()
print(crypto_df.shape)
crypto_df.head()

(685, 6)


Unnamed: 0   CoinName Algorithm ProofType  TotalCoinsMined TotalCoinSupply
0         42    42 Coin    Scrypt   PoW/PoS     4.199995e+01              42
2        404    404Coin    Scrypt   PoW/PoS     1.055185e+09       532000000
4        808        808   SHA-256   PoW/PoS     0.000000e+00               0
5       1337  EliteCoin       X13   PoW/PoS     2.927942e+10    314159265359
7        BTC    Bitcoin   SHA-256       PoW     1.792718e+07        21000000

In [250]:
# Keep the rows where coins are mined.
crypto_df = crypto_df[crypto_df['TotalCoinsMined'] > 0]
crypto_df.head()


Unnamed: 0   CoinName Algorithm ProofType  TotalCoinsMined TotalCoinSupply
0         42    42 Coin    Scrypt   PoW/PoS     4.199995e+01              42
2        404    404Coin    Scrypt   PoW/PoS     1.055185e+09       532000000
5       1337  EliteCoin       X13   PoW/PoS     2.927942e+10    314159265359
7        BTC    Bitcoin   SHA-256       PoW     1.792718e+07        21000000
8        ETH   Ethereum    Ethash       PoW     1.076842e+08               0

In [251]:
print(crypto_df.shape)

(532, 6)


In [252]:
crypto_df.columns

Index(['Unnamed: 0', 'CoinName', 'Algorithm', 'ProofType', 'TotalCoinsMined',
       'TotalCoinSupply'],
      dtype='object')

In [253]:
crypto_df.dtypes

Unnamed: 0          object
CoinName            object
Algorithm           object
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply     object
dtype: object

In [254]:
coin_names = crypto_df["CoinName"].values
Index = crypto_df["Unnamed: 0"]


In [255]:
# Create a new DataFrame that holds only the cryptocurrencies names.
coin_name_df = pd.DataFrame(coin_names, index=Index)
##coin_name_df.rename(columns={"Unnamed: 0": "Index", "0": "CoinName"}, inplace=True)
coin_name_df.head(10)

0
Unnamed: 0                  
42                   42 Coin
404                  404Coin
1337               EliteCoin
BTC                  Bitcoin
ETH                 Ethereum
LTC                 Litecoin
DASH                    Dash
XMR                   Monero
ETC         Ethereum Classic
ZEC                    ZCash

In [256]:
crypto_df = crypto_df.set_index("Unnamed: 0")
crypto_df.head()


CoinName Algorithm ProofType  TotalCoinsMined TotalCoinSupply
Unnamed: 0                                                                
42            42 Coin    Scrypt   PoW/PoS     4.199995e+01              42
404           404Coin    Scrypt   PoW/PoS     1.055185e+09       532000000
1337        EliteCoin       X13   PoW/PoS     2.927942e+10    314159265359
BTC           Bitcoin   SHA-256       PoW     1.792718e+07        21000000
ETH          Ethereum    Ethash       PoW     1.076842e+08               0

In [257]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df.drop(['CoinName'], inplace=True, axis = 1)

print(crypto_df.shape)
crypto_df.head()

(532, 4)


Algorithm ProofType  TotalCoinsMined TotalCoinSupply
Unnamed: 0                                                     
42            Scrypt   PoW/PoS     4.199995e+01              42
404           Scrypt   PoW/PoS     1.055185e+09       532000000
1337             X13   PoW/PoS     2.927942e+10    314159265359
BTC          SHA-256       PoW     1.792718e+07        21000000
ETH           Ethash       PoW     1.076842e+08               0

In [258]:
print(crypto_df.shape)

(532, 4)


In [259]:
# Use get_dummies() to create variables for text features.
crypto_df_features = pd.get_dummies(crypto_df, columns=['Algorithm', 'ProofType'])

crypto_df_features.head()

TotalCoinsMined TotalCoinSupply  Algorithm_1GB AES Pattern Search  \
Unnamed: 0                                                                      
42             4.199995e+01              42                                 0   
404            1.055185e+09       532000000                                 0   
1337           2.927942e+10    314159265359                                 0   
BTC            1.792718e+07        21000000                                 0   
ETH            1.076842e+08               0                                 0   

            Algorithm_536  Algorithm_Argon2d  Algorithm_BLAKE256  \
Unnamed: 0                                                         
42                      0                  0                   0   
404                     0                  0                   0   
1337                    0                  0                   0   
BTC                     0                  0                   0   
ETH                     0                  0                   0   

            Algorithm_Blake  Algorithm_Blake2S  Algorithm_Blake2b  \
Unnamed: 0                                                          
42                        0                  0                  0   
404                       0                  0                  0   
1337                      0                  0                  0   
BTC                       0                  0                  0   
ETH                       0                  0                  0   

            Algorithm_C11  ...  ProofType_PoW/PoS  ProofType_PoW/PoS   \
Unnamed: 0                 ...                                          
42                      0  ...                  1                   0   
404                     0  ...                  1                   0   
1337                    0  ...                  1                   0   
BTC                     0  ...                  0                   0   
ETH                     0  ...                  0                   0   

            ProofType_PoW/PoW  ProofType_PoW/nPoS  ProofType_Pos  \
Unnamed: 0                                                         
42                          0                   0              0   
404                         0                   0              0   
1337                        0                   0              0   
BTC                         0                   0              0   
ETH                         0                   0              0   

            ProofType_Proof of Authority  ProofType_Proof of Trust  \
Unnamed: 0                                                           
42                                     0                         0   
404                                    0                         0   
1337                                   0                         0   
BTC                                    0                         0   
ETH                                    0                         0   

            ProofType_TPoS  ProofType_Zero-Knowledge Proof  ProofType_dPoW/PoW  
Unnamed: 0                                                                      
42                       0                               0                   0  
404                      0                               0                   0  
1337                     0                               0                   0  
BTC                      0                               0                   0  
ETH                      0                               0                   0  

[5 rows x 98 columns]

In [260]:
print(crypto_df_features.shape)

(532, 98)


In [261]:
crypto_scaled = StandardScaler().fit_transform(crypto_df_features)
print(crypto_scaled)

[[-0.11710817 -0.1528703  -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.09396955 -0.145009   -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [ 0.52494561  4.48942416 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 ...
 [-0.09561336 -0.13217937 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.11694817 -0.15255998 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.11710536 -0.15285552 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]]


### Deliverable 2: Reducing Data Dimensions Using PCA

In [262]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(crypto_scaled)
print(crypto_pca)

[[-0.34415473  0.85202548 -0.58128166]
 [-0.32754267  0.85122191 -0.58169071]
 [ 2.29072913  1.36016745 -0.63187247]
 ...
 [ 0.33183428 -2.22371696  0.43122258]
 [-0.13627916 -1.93648228  0.41037482]
 [-0.29230063  0.83189209 -0.26777881]]


In [263]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(data=crypto_pca, columns=["Principle Component 1", "Principle Component 2", "Principle Component 3"], index=crypto_df.index)

print(pcs_df.shape)
pcs_df.head()

(532, 3)


Principle Component 1  Principle Component 2  \
Unnamed: 0                                                 
42                      -0.344155               0.852025   
404                     -0.327543               0.851222   
1337                     2.290729               1.360167   
BTC                     -0.135786              -1.205484   
ETH                     -0.146877              -1.885776   

            Principle Component 3  
Unnamed: 0                         
42                      -0.581282  
404                     -0.581691  
1337                    -0.631872  
BTC                      0.195634  
ETH                      0.383749

In [264]:
pca.explained_variance_ratio_

array([0.0279309 , 0.02132839, 0.02051005])

### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [265]:
# Create an elbow curve to find the best value for K.
# Find the best value for K
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")


C:\Users\Mama\anaconda3\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:1039: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [266]:
# Initialize the K-Means model.
model = KMeans(n_clusters=2, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)
print(predictions)

[0 0 0 3 3 3 0 3 3 3 0 3 0 0 3 0 3 3 0 0 3 3 3 3 3 0 3 3 3 0 3 0 3 3 0 0 3
 3 3 3 3 3 0 0 3 3 3 3 3 0 0 3 0 3 3 3 3 0 3 3 0 3 0 0 0 3 3 3 0 0 0 0 0 3
 3 3 0 0 3 0 3 0 0 3 3 3 3 0 0 3 0 3 3 0 0 3 0 0 3 3 0 0 3 0 0 3 0 3 0 3 0
 3 0 0 3 3 0 3 3 3 0 3 3 3 3 3 0 0 3 3 3 0 3 0 3 3 0 3 0 3 0 0 3 3 0 3 3 0
 0 3 0 3 0 0 0 3 3 3 3 0 0 0 0 0 3 3 0 0 0 0 0 3 0 0 0 0 0 3 0 3 0 0 3 0 3
 0 0 3 0 3 0 3 0 3 0 0 0 0 3 0 0 0 0 0 3 3 0 0 3 3 0 0 0 0 0 3 0 0 0 0 0 0
 0 0 3 0 0 0 0 0 0 3 3 3 0 0 0 0 3 0 3 0 0 3 0 3 3 0 3 3 0 3 0 0 0 3 0 0 3
 0 0 0 0 0 0 0 3 0 3 0 0 0 0 3 0 3 0 3 3 3 3 0 3 0 0 3 0 3 3 3 0 3 0 3 3 3
 0 3 0 3 0 0 0 3 0 3 3 3 3 3 0 0 3 0 0 0 3 0 3 0 3 0 3 0 0 0 0 3 0 0 3 0 0
 0 3 3 3 3 0 0 0 0 3 0 3 3 3 0 0 3 3 0 0 3 0 3 3 3 0 3 3 0 0 0 3 3 3 0 0 0
 3 3 0 3 3 3 3 0 1 1 3 3 3 0 1 0 0 0 0 3 3 3 3 0 0 0 3 0 3 0 0 0 0 3 0 0 3
 0 0 3 3 0 3 0 3 3 3 3 0 0 3 0 3 0 0 0 0 0 0 3 3 3 0 0 0 0 0 0 3 0 3 3 3 3
 0 0 0 0 3 0 0 3 0 0 3 1 3 0 3 3 0 0 3 0 3 3 0 3 3 0 3 0 3 0 0 3 0 0 0 0 0
 3 3 3 0 0 0 3 0 3 0 3 0 

In [267]:
pcs_df.head()

Principle Component 1  Principle Component 2  \
Unnamed: 0                                                 
42                      -0.344155               0.852025   
404                     -0.327543               0.851222   
1337                     2.290729               1.360167   
BTC                     -0.135786              -1.205484   
ETH                     -0.146877              -1.885776   

            Principle Component 3  
Unnamed: 0                         
42                      -0.581282  
404                     -0.581691  
1337                    -0.631872  
BTC                      0.195634  
ETH                      0.383749

In [268]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
frames = [crypto_df, pcs_df]
clustered_df=pd.concat(frames, axis=1)
clustered_df.head()

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
frames = [clustered_df, coin_name_df]
clustered_df=pd.concat(frames, axis=1)

clustered_df.rename(columns={0:"CoinName"}, inplace=True)

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


Algorithm ProofType  TotalCoinsMined TotalCoinSupply  \
Unnamed: 0                                                              
42                  Scrypt   PoW/PoS     4.199995e+01              42   
404                 Scrypt   PoW/PoS     1.055185e+09       532000000   
1337                   X13   PoW/PoS     2.927942e+10    314159265359   
BTC                SHA-256       PoW     1.792718e+07        21000000   
ETH                 Ethash       PoW     1.076842e+08               0   
LTC                 Scrypt       PoW     6.303924e+07        84000000   
DASH                   X11   PoW/PoS     9.031294e+06        22000000   
XMR         CryptoNight-V7       PoW     1.720114e+07               0   
ETC                 Ethash       PoW     1.133597e+08       210000000   
ZEC               Equihash       PoW     7.383056e+06        21000000   

            Principle Component 1  Principle Component 2  \
Unnamed: 0                                                 
42                      -0.344155               0.852025   
404                     -0.327543               0.851222   
1337                     2.290729               1.360167   
BTC                     -0.135786              -1.205484   
ETH                     -0.146877              -1.885776   
LTC                     -0.166527              -1.152696   
DASH                    -0.403997               1.095522   
XMR                     -0.138788              -2.113966   
ETC                     -0.145323              -1.885967   
ZEC                     -0.136278              -1.936482   

            Principle Component 3          CoinName  Class  
Unnamed: 0                                                  
42                      -0.581282           42 Coin      0  
404                     -0.581691           404Coin      0  
1337                    -0.631872         EliteCoin      0  
BTC                      0.195634           Bitcoin      3  
ETH                      0.383749          Ethereum      3  
LTC                     -0.010239          Litecoin      3  
DASH                    -0.546040              Dash      0  
XMR                      0.440304            Monero      3  
ETC                      0.383731  Ethereum Classic      3  
ZEC                      0.410375             ZCash      3

### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [269]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(clustered_df, x="Principle Component 1", y="Principle Component 2", z="Principle Component 3", color="Class", symbol="Class",width=800, hover_name="CoinName", hover_data=["Algorithm"])
fig.update_layout(legend=dict(x=0,y=1))
fig.show()

In [270]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=["CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "Class"], sortable=True, selectable = True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [271]:
# Print the total number of tradable cryptocurrencies.
my_set = set(clustered_df["CoinName"])
unique_vals = len(my_set)
print(f"There are {unique_vals} unique cryptocurrencies.")

There are 531 unique cryptocurrencies.


In [272]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
data = clustered_df[['TotalCoinSupply', 'TotalCoinsMined']] 

data_scaled = MinMaxScaler().fit_transform(data)
print(data_scaled[0:5])

[[4.20000000e-11 0.00000000e+00]
 [5.32000000e-04 1.06585544e-03]
 [3.14159265e-01 2.95755135e-02]
 [2.10000000e-05 1.81084216e-05]
 [0.00000000e+00 1.08773140e-04]]


In [279]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = pd.DataFrame(data_scaled)
##plot_df.set_index(coin_name_df["Unnamed: 0"])
plot_df.rename(columns={0:"TotalCoinSupply", 1: "TotalCoinsMined"}, inplace=True)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
#plot_df = pd.merge(plot_df, clustered_df["Class"])
added_data = pd.concat([clustered_df["CoinName"], clustered_df["Class"]])

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 

plot_df = pd.concat([plot_df, added_data], axis =1)
 

plot_df.head(10)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [278]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
# YOUR CODE HERE
plot_df.hvplot.scatter(
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    hover_cols=["class"],
    by="class",
)

DataError: Supplied data does not contain specified dimensions, the following dimensions were not found: ['class', 'TotalCoinsMined', 'TotalCoinSupply']

PandasInterface expects tabular data, for more information on supported datatypes see http://holoviews.org/user_guide/Tabular_Datasets.html